# EC1B1 Group project


In [118]:
# Downloading the data

import requests
import pandas as pd
from io import StringIO

# Google Sheets details
SHEET_ID = "1VnzB2MO5jEBzXqtkA0i_oNtONzOIg9y3I5nU1fB6DwU"
# List of sheets and their GIDs (Find these manually or automate if needed)
SHEET_GIDS = {
    "Sheet1": "0",  
    "Sheet2": "1811506178",
    "Sheet3": "1443003215"
}

dfs = {}

# Loop through each sheet and fetch data
for sheet_name, gid in SHEET_GIDS.items():
    url = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={gid}"
    response = requests.get(url)

    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))  # Convert CSV text to DataFrame
        dfs[sheet_name] = df
        print(f"Extracted: {sheet_name} ✅")
    else:
        print(f"Failed to extract {sheet_name}, Status Code: {response.status_code}")

# Display extracted DataFrames
#print("Extracted Sheets:", dfs.keys())

# Example: Print first few rows of a sheet
#print(dfs["Sheet2"].head())  # Change "Sheet1" to the actual sheet name

germany_data=dfs["Sheet1"]
us_data=dfs["Sheet2"]
data=dfs['Sheet3']


Extracted: Sheet1 ✅
Extracted: Sheet2 ✅
Extracted: Sheet3 ✅


In [ ]:
# processing germany data
df = germany_data.drop(columns='Germany').T
df.columns = df.iloc[0]  # Set the first row as column headers
df = df[1:]
df["Country"] = "Germany"
germany_df = df

In [ ]:
# processing us data
df2 = us_data.drop(columns='USA').T
df2.columns = df2.iloc[0]  # Set the first row as column headers
df2 = df2[1:]
df2["Country"] = "USA"
us_df = df2

In [143]:

df = us_data.drop(columns='USA')

df.loc[-1] = df.columns  # Insert the column names as a row at index -1
df.index = df.index + 1  # Shift index to start from 1 to maintain the order
df = df.sort_index()  # Sort the DataFrame by index
df.columns = range(df.shape[1])
newdf = df.T
# df.columns = df.iloc[0]

newdf.columns = newdf.iloc[0]

newdf = newdf.drop(0)

newdf["Country"] = "USA"

us_df = newdf

In [ ]:
# cont processing data as a whole

df = data
# Convert column headers into a row
df = pd.DataFrame(df.values, columns=["Header"] * len(df.columns))

# Add the original column names as a new row
df.loc[-1] = df.columns.tolist()

# Reset index to make the new row the first row
df = df.reset_index(drop=True)

# Print the result
df
# cont processing data as a whole

data.T


In [ ]:
germany_df


In [146]:
# processing germany data

def convert_to_int(value):
    if isinstance(value, str) and 'M' in value:
        value = value.replace(',', '')  # Remove commas
        return int(float(value.replace('M', '').strip()) * 1_000_000)
    return value  # Return as is if not in expected format

# Apply the function
germany_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] = germany_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].apply(convert_to_int)

# germany_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, Gold (Including Gold Deposits and, If Appropriate, Gold Swapped), US Dollar'] = germany_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, Gold (Including Gold Deposits and, If Appropriate, Gold Swapped), US Dollar'].apply(convert_to_int)

germany_df
us_df

us_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] = us_df['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].apply(convert_to_int)
#print(us_df.columns)



In [147]:

us_df

,"International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar","Prices, Consumer Price Index, All items, Index",Country
1,21504500000,13.48,USA
2,21478100000,13.44,USA
3,21395700000,13.48,USA
4,21344700000,13.48,USA
5,21278000000,13.53,USA
6,21234300000,13.53,USA
7,21169800000,13.57,USA
8,20999400000,13.57,USA
9,20803400000,13.57,USA
10,20430000000,13.57,USA


data has been processed into us_df anf germany_df with the cleaned data

In [150]:
germany_df.to_csv('germany_clean_data.csv', index=False)
us_df.to_csv('us_clean_data.csv', index=False)